개발 환경

DISTRIB_ID=Ubuntu

DISTRIB_RELEASE=18.04

DISTRIB_CODENAME=bionic

DISTRIB_DESCRIPTION="Ubuntu 18.04.5 LTS"

라이브러리 버전

albumentations==1.1.0

opencv-python==4.5.3.56

glob2==0.7

Pillow==8.3.2

tensorboard == 2.7.0

timm==0.4.12

torch==1.8.1

tqdm==5.62.3

In [ ]:
# 필요 파일 설치 부분
# !pip install glob2
# !pip install opencv-python
# !pip install albumentations
# !pip install Pillow
# !pip install tensorboard
# !pip install timm
# !pip install tqdm

먼저 필요한 라이브러리와 프레임워크를 미리 불러들이겠습니다.

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from itertools import combinations
from PIL import Image

import os
import re
import csv
import glob
import cv2
import shutil
import tqdm
import albumentations
import albumentations.pytorch as albu_torch
import torch
import torch.nn as nn


우선, 훈련용 데이터 셋과 검증용 테스트 셋을 나누고, 검증용 데이터 셋과 테스트 셋의 경우 추론에 적합하도록 이미지 크기를 줄이도록 하겠습니다.

이미지 각각의 크기가 매우 커서 이미지 로드와 리사이즈에 매우 시간이 많이 걸리는데, 불필요한 시간을 줄이기 위함입니다.

다만, 추후 어그멘테이션 방법 때문에 훈련용 데이터 셋의 경우 리사이즈를 하지 않고 저장합니다.

그리고 대회 참여를 위해 사용해야하는 test_data.csv 파일이 test_dataset에 있습니다만, glob을 통해서 쉽게 이미지를 추출하기 위해 빼도록 폴더 밖으로 빼도록 하겠습니다.

In [ ]:
# this root is important as the DACON require this part. but if you unzip the data file(.zip), the name is "open", just change this path.
BASE_DATA_ROOT = '/data'
# default folder name
TEST_DATA_ROOT = 'test_dataset'
TRAIN_VAL_DATA_ROOT = '/train_dataset'
# validation set
VAL_DATA_FOLDERS = ('BC_09', 'LT_10')
TEST_CSV_NAME = 'test_data.csv'
shutil.move(os.path.join(BASE_DATA_ROOT, TEST_DATA_ROOT, TEST_CSV_NAME), os.path.join(BASE_DATA_ROOT, TEST_CSV_NAME))
# resize test set files all
test_set_files = glob.glob(os.path.join(BASE_DATA_ROOT, TEST_DATA_ROOT, '*', '*', '*'))
for file_path in tqdm.tqdm(test_set_files):
    img = Image.open(file_path)
    img = img.resize((448, 448))
    img.save(file_path)

# resize validation set files only
train_val_set_files = glob.glob(os.path.join(BASE_DATA_ROOT, TRAIN_VAL_DATA_ROOT, '*', '*', '*'))
for file_path in tqdm.tqdm(train_val_set_files):
    for val_data_folder_name in VAL_DATA_FOLDERS:
        if val_data_folder_name in file_path:
            img = Image.open(file_path)
            img = img.resize((448, 448))
            img.save(file_path)

데이터 셋에 대한 준비는 이것으로 간단히 끝났습니다.

모델 아키텍처를 구성해보도록 하겠습니다.

timm 이라는 라이브러리를 통해서 이미 훈련된 모델을 불러들일 수 있습니다.

저는 Big Transfer(https://arxiv.org/abs/1912.11370)라는 논문에서 소개된 모델을 이용하겠습니다. 다른 SWIN transformer와 같은 모델도 고려하였지만, 파라미터 튜닝까지 다 할 것을 고려할 때, 현실적으로 불가능하였습니다.그리고 당연히 모델이 크면 클수록 좋지만, GPU 성능의 한계 때문에 최대 크기를 사용하지는 못했습니다.

모델 구성의 아이디어를 간략히 설명하자면, 우선 timm 라이브러리에서 제공하는 이미 훈련된 모델을 사용하였습니다. Big Tranfer라는 모델로, 전이학습을 위해서 다양한 모델구조와 방법을 시도했던 논문으로 기존에 300만장의 데이터에 pretrain된 후, 다시한번 이를 448x448 이미지 사이즈로 1000개의 카테고리에 다시한번 fine tuning한 모델을 사용하였습니다.

모델 아키텍처를 구성할 때, 제가 생각하기에 사람의 성장 정도를 파악할 때 사람의 키나 골격, 얼굴 형태 등 다양한 요소를 종합하여 판단내리듯, 식물이 성장한 시간을 추측하기 위해서도 다양한 feature로부터 계산을 이끌어내야한다고 판단하였습니다.

따라서 공통된 feature를 추출하기 위해서 단일의 pretrained 모델을 사용하되, 이후 다양한 features를 concat을 하여 이후 연산을 진행하고자 하였습니다. 또한 이떄도 여러 번의 연산을 거치면(깊이가 깊어지면) 정확도가 높아질 거라 예상하고 깊이를 추가하였습니다.

마지막으로 pretrained 모델을 로드할 때, pretrained 당시에 사용했던 이미지 크기와 hearder의 weight를 그대로 사용하였습니다. 이는 훈련 시간을 축소시키면서도 기존에 1000가지의 category에 이미 학습되었다면 충분히 feature로 활용할 수 있을 거라 가정하였습니다.(컴퓨터 성능 문제인지 시간 압박이 상당했습니다.)

In [ ]:
from timm.data.transforms_factory import create_transform
from PIL import Image
import timm
import torch
import torch.nn as nn

class PlantAgePrediction(nn.Module):
    def __init__(self, drop_p=0.3):
        super().__init__()
        self.base_model = timm.create_model('resnetv2_152x2_bitm', pretrained=True, drop_rate=drop_p)
        self.activation = nn.GELU()
        
        self.drop = nn.Dropout(drop_p)
        self.fc1 = nn.Linear(2000, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 200)
        self.fc4 = nn.Linear(200, 1)
        
    def forward(self, young, old):
        young = self.drop(self.activation(self.base_model(young)))
        old = self.drop(self.activation(self.base_model(old)))
        
        total = torch.cat((young, old), dim=1)
        total = self.drop(self.activation(self.fc1(total)))
        total = self.drop(self.activation(self.fc2(total)))
        total = self.drop(self.activation(self.fc3(total)))
        total = self.fc4(total)
        
        return total

이후 실제로 훈련하기 위해 이미지 어그멘테이션과 데이터 normalize, 데이터 로드를 위한 데이터 셋을 준비하겠습니다.

이미지 어그멘테이션의 경우, 우선 rotation과 flip을 준 이후 다양한 크기로 crop하였고, 유일한 규칙은 아무리 작아도 식물의 한 부분은 보여야 한다는 점과 crop과 flip이 두 이미지 모두 동일해야한다는 것입니다.
이는 동일한 대상에게 특징적인 feature, 예를 들어 잎사귀의 형태, 해상도가 높은 경우 잎맥의 모습, 색상 등, 사람에게 명확하지 않더라도 모델이 구분하고 그를 기준으로 식물이 성장한 시간을 추론할 수 있지 않을까 가설을 세웠기 떄문입니다. 또한 마찬가지로 동일한 대상의 성장을 추측하는 것이 의미있기에, 같은 대상 파일에서만 성장 수준을 비교하도록 하였습니다.

이때, 대회측에서 설명한 데이터 투입 구조는 항상 before - after 의 시간적 순서를 따르게 되므로, 모델 학습시에도 이런 구조를 따르도록 하며, 데이터 셋 구성 또한 이를 따르도록 만들었습니다.
normalize의 경우, 두 식물간의 시간차가 최대 42까지 나는 것으로 확인되었는데, 이러한 큰 값은 학습을 불안정하게 만들 수 있어 값을 축소 시키기 위한 방법으로 시행하였습니다. 특히, 결과값이 1인 경우는 매우 많으나, 결과값이 커질수록 데이터의 양이 매우 줄어드는 문제가 있었습니다. 따라서, min-max normalize를 진행하였습니다.

In [ ]:
# augmentation for validation and test
def only_resize():
    return albumentations.Compose([
        albumentations.Resize(448, 448, always_apply=True),
        albumentations.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), always_apply=True),
        albu_torch.transforms.ToTensorV2()
    ], additional_targets={'young': 'image', 'old': 'image'})

# augmentation for train
def only_rotate():
    return albumentations.Compose([
        albumentations.Flip(p=0.5),
        albumentations.Rotate(limit=180, always_apply=True),
        albumentations.augmentations.crops.transforms.RandomResizedCrop(448, 448, scale=(0.2, 1), ratio=(1, 1), always_apply=True),
        albumentations.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), always_apply=True),
        albu_torch.transforms.ToTensorV2()
    ], additional_targets={'young': 'image', 'old': 'image'})

def image_load_only(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def image_agmentation_apply(young, old, augmantation):
    image_dict = {'image': young, 'old': old}
    aug_dict = augmantation(**image_dict)
    return aug_dict['image'], aug_dict['old']

def denormalize(predicted, min_value, max_value):
    output = predicted * (max_value - min_value) + min_value
    return output

저의 경우는 데이터를 미리 다 불러들임으로써 반복적인 데이터 로드로 인한 부하를 줄였습니다. 다만, 충분한 RAM 이 없는 분을 위해 그때 그때 데이터를 불러들이는 데이터셋도 적어둡니다.

In [ ]:
class PlantDataset(Dataset):
    def __init__(self, base_folder_path, train=True):
        super().__init__()
        kinds = os.listdir(base_folder_path) # BC, LT
        self.data = []
        preload_img = {}
        
        for kind in kinds:
            target_plant_kind_number_path = os.path.join(base_folder_path, kind)
            target_plant_kind_number_images = os.listdir(target_plant_kind_number_path)
            for target_plant_number in target_plant_kind_number_images:
                if train and target_plant_number not in VAL_DATA_FOLDERS:
                    target_plant_kind_number_images_path = os.path.join(target_plant_kind_number_path, target_plant_number)
                    target_plant_images = sorted(os.listdir(target_plant_kind_number_images_path))
                    target_plant_images_pair = list(combinations(target_plant_images, 2))
                    for img_pair in target_plant_images_pair:
                        young, old = img_pair
                        young_path = os.path.join(target_plant_kind_number_images_path, young)
                        old_path = os.path.join(target_plant_kind_number_images_path, old)
                        young = int(re.findall(r'[0-9]+', young)[-1])
                        old = int(re.findall(r'[0-9]+', old)[-1])
                        if young_path not in preload_img:
                            preload_img[young_path] = image_load_only(young_path)
                        if old_path not in preload_img:
                            preload_img[old_path] = image_load_only(old_path)
                        
                        self.data.append({'data': [preload_img[young_path], preload_img[old_path]], 'label': old-young})
                        
                elif not train and target_plant_number in VAL_DATA_FOLDERS:
                    target_plant_kind_number_images_path = os.path.join(target_plant_kind_number_path, target_plant_number)
                    target_plant_images = sorted(os.listdir(target_plant_kind_number_images_path))
                    target_plant_images_pair = list(combinations(target_plant_images, 2))
                    for img_pair in target_plant_images_pair:
                        young, old = img_pair
                        young_path = os.path.join(target_plant_kind_number_images_path, young)
                        old_path = os.path.join(target_plant_kind_number_images_path, old)
                        young = int(re.findall(r'[0-9]+', young)[-1])
                        old = int(re.findall(r'[0-9]+', old)[-1])
                        if young_path not in preload_img:
                            preload_img[young_path] = image_load_only(young_path)
                        if old_path not in preload_img:
                            preload_img[old_path] = image_load_only(old_path)
                        
                        self.data.append({'data': [preload_img[young_path], preload_img[old_path]], 'label': old-young})
                
        if train:
            self.augmentation = only_rotate()
        else:
            self.augmentation = only_resize()
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        young_path, old_path = self.data[index]['data']
        young_img, old_img = image_agmentation_apply(young_path, old_path, self.augmentation)
        
        return young_img, old_img, torch.tensor(self.data[index]['label'], dtype=torch.float).unsqueeze(-1)

    def _minmax_scaling(self):
        labels = [one_dict['label'] for one_dict in self.data]
        min_value = min(labels)
        max_value = max(labels)
        print(f'min: {min_value}, max: {max_value}')
        for one_dict in self.data:
            one_dict['label'] = (one_dict['label'] - min_value) / (max_value - min_value)
            
        return min_value, max_value
        
    def _get_minmax_scaling(self, min_value, max_value):
        for one_dict in self.data:
            one_dict['label'] = (one_dict['label'] - min_value) / (max_value - min_value)
        


In [ ]:
# when there's not enough RAM
# def image_load(img_path_young, img_path_old, augmantation):
#     young = cv2.imread(img_path_young)
#     young = cv2.cvtColor(young, cv2.COLOR_BGR2RGB)
#     old = cv2.imread(img_path_old)
#     old = cv2.cvtColor(old, cv2.COLOR_BGR2RGB)
#     image_dict = {'image': young, 'old': old}
#     aug_dict = augmantation(**image_dict)
#     return aug_dict['image'], aug_dict['old']

# class PlantDataset(Dataset):
#     def __init__(self, base_folder_path, train=True):
#         super().__init__()
#         kinds = os.listdir(base_folder_path) # BC, LT
#         self.data = []
        
#         for kind in kinds:
#             target_plant_kind_number_path = os.path.join(base_folder_path, kind)
#             target_plant_kind_number_images = os.listdir(target_plant_kind_number_path)
#             for target_plant_number in target_plant_kind_number_images:
#                 if train and target_plant_number not in VAL_DATA_FOLDERS:
#                     target_plant_kind_number_images_path = os.path.join(target_plant_kind_number_path, target_plant_number)
#                     target_plant_images = sorted(os.listdir(target_plant_kind_number_images_path))
#                     target_plant_images_pair = list(combinations(target_plant_images, 2))
#                     for img_pair in target_plant_images_pair:
#                         young, old = img_pair
#                         young_path = os.path.join(target_plant_kind_number_images_path, young)
#                         old_path = os.path.join(target_plant_kind_number_images_path, old)
#                         young = int(re.findall(r'[0-9]+', young)[-1])
#                         old = int(re.findall(r'[0-9]+', old)[-1])
#                         self.data.append({'data': [young_path, old_path], 'label': old-young})
                        
#                 elif not train and target_plant_number in VAL_DATA_FOLDERS:
#                     target_plant_kind_number_images_path = os.path.join(target_plant_kind_number_path, target_plant_number)
#                     target_plant_images = sorted(os.listdir(target_plant_kind_number_images_path))
#                     target_plant_images_pair = list(combinations(target_plant_images, 2))
#                     for img_pair in target_plant_images_pair:
#                         young, old = img_pair
#                         young_path = os.path.join(target_plant_kind_number_images_path, young)
#                         old_path = os.path.join(target_plant_kind_number_images_path, old)
#                         young = int(re.findall(r'[0-9]+', young)[-1])
#                         old = int(re.findall(r'[0-9]+', old)[-1])
#                         self.data.append({'data': [young_path, old_path], 'label': old-young})
                
#         if train:
#             self.augmentation = only_rotate()
#         else:
#             self.augmentation = only_resize()
            
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         young_path, old_path = self.data[index]['data']
#         young_img, old_img = image_load(young_path, old_path, self.augmentation)
        
#         return young_img, old_img, torch.tensor(self.data[index]['label'], dtype=torch.float).unsqueeze(-1)
    
#     def _minmax_scaling(self):
#         labels = [one_dict['label'] for one_dict in self.data]
#         min_value = min(labels)
#         max_value = max(labels)
#         print(f'min: {min_value}, max: {max_value}')
#         for one_dict in self.data:
#             one_dict['label'] = (one_dict['label'] - min_value) / (max_value - min_value)
            
#         return min_value, max_value
        
#     def _get_minmax_scaling(self, min_value, max_value):
#         for one_dict in self.data:
#             one_dict['label'] = (one_dict['label'] - min_value) / (max_value - min_value)
    

다음으로 실제 훈련을 위한 코드입니다.

특징으로는 AdamW를 사용했고, 공식적으로는 RMSE를 기준으로 채점한다고 하였습니다만, 현재 준비된 데이터 셋이 normalize가 되었을 뿐만 아니라, 오차가 클 수 있는 데이터 예측치일 수록 데이터 수량이 적어진다는 것을 고려할 때, 오히려 이러한 불균형을 고려하는 것이 중요할 것으로 생각하였습니다.

그러면 이러한 불균형에 좀더 가중치를 줄 수 있는 loss function이 필요할 것으로 생각되었고, 동시에 오히려 빈발하는 데이터 결과 값에 대해서는 낮은 가중치를 주는게 더 좋은 결과를 내지 않을까 생각하였습니다. 그 결과 저는 MSE를 loss function으로 사용하였습니다.

그리고 tensorboard를 통해 실시간으로 관찰하였고, 과적합이 시사될 때 훈련을 중단하였습니다.

사용법은 커맨드 창에서

tensorboard --logdir=[폴더경로]

이렇게 하면 되겠습니다. 아래 코드에서는 "log_dir"을 기본값으로 주었습니다.

In [ ]:
# Total epochs is not important. I always checked the training graph, and if there's over fitting sign, I stopped.
EPOCHS = 10000
# This Batch size is not optimal. I want to try more large batch size, but my GPU.... sadly.... 
BATCH_SIZE = 12
# Almost only hyperparameter that I can test. This was best in this setting.
DROPOUT_RATE = 0.3
LEARNING_RATE = 0.00001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LOG_DIR = "log_dir"
DATASET_FOLDER = os.path.join(BASE_DATA_ROOT, TRAIN_VAL_DATA_ROOT)
SAVE_FOLDER = 'save_model'
os.makedirs(SAVE_FOLDER, exist_ok=True)

print('Train data starts to load.')
train_dataset = PlantDataset(DATASET_FOLDER, train=True)
MINMAX_MIN, MINMAX_MAX = train_dataset._minmax_scaling()
print('Test data starts to load.')
val_dataset = PlantDataset(DATASET_FOLDER, train=False)
val_dataset._get_minmax_scaling(MINMAX_MIN, MINMAX_MAX)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, prefetch_factor=BATCH_SIZE*2, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, prefetch_factor=BATCH_SIZE*2, pin_memory=True)

print('Model starts to load.')
model = PlantAgePrediction(drop_p=DROPOUT_RATE)
model = torch.nn.DataParallel(model).to(DEVICE)
print('Model is loaded.')
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# This is to continue training.
# checkpoint = torch.load('save_model_temp/latest.pt', map_location=DEVICE)
# for key in list(checkpoint.keys()):
#     if 'module.' in key:
#         checkpoint[key.replace('module.', '')] = checkpoint[key]
#         del checkpoint[key]
# model.load_state_dict(checkpoint['model'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# del checkpoint
loss_fn = nn.MSELoss()
writer = SummaryWriter(LOG_DIR)

step = 0
loss_min = float('inf')
for epoch in range(1, EPOCHS+1):
    model.train()
    for data in tqdm.tqdm(train_dataloader):
        young, old, labels = data
        young = young.to(DEVICE)
        old = old.to(DEVICE)
        labels = labels.to(DEVICE)
        
        optimizer.zero_grad()
        logits = model(young, old)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        step += 1
        writer.add_scalar('Loss/train_step', loss, step)
    
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in tqdm.tqdm(val_dataloader):
            young, old, labels = data
            young = young.to(DEVICE)
            old = old.to(DEVICE)
            labels = labels.to(DEVICE)
            logits = model(young, old)
            total_loss += loss_fn(logits, labels) * len(labels)
            
    if loss_min > total_loss:
        loss_min = total_loss
        torch.save(model.state_dict(), SAVE_FOLDER + '/min_loss.pt')
    torch.save(model.state_dict(), SAVE_FOLDER + f'/{epoch}.pt')
    torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict()}, SAVE_FOLDER + f'/latest.pt')
    writer.add_scalar('Loss/val', total_loss/len(val_dataset), epoch)

마지막으로 충분히 훈련이 진행되었을 때, 테스트 셋을 불러와서 csv 에 결과를 산출하게 됩니다.

이때, normalized 값은 산출하도록 훈련되었기에, 이를 원래 값으로 복원해야만 합니다.

또한 후처리가 진행되는데, 모델이 산출하는 결과는 음수가 나올 수 있습니다만, csv 형식 상, 항상 데이터가 투입되는 구조는 before_after 형식을 따르게 되므로, 항상 0 이상의 값이며, 데이터 구조상 논리적으로 최저 1을 주는 것이 합당하다고 생각하여 1보다 작은 값이 산출될 경우 일괄적으로 1로 변환하였습니다.

진행하기에 앞서, 데이터 셋 파일이 RAM을 매우 많이 차지하고 있기에 이를 제거해둡니다.

In [ ]:
del train_dataset
del val_dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_path_for_glob, csv_path):
        target_files = glob.glob(img_path_for_glob)
        self.data = []
        preload_img = {}
        with open(csv_path, 'r', encoding='utf-8') as f:
            csv_reader = csv.reader(f)
            csv_reader = list(csv_reader)
            for line in csv_reader[1:]:
                _, young, old = line
                for file_name in target_files:
                    if young in file_name:
                        young_path = file_name
                    if old in file_name:
                        old_path = file_name
                if young_path not in preload_img:
                    preload_img[young_path] = image_load_only(young_path)
                if old_path not in preload_img:
                    preload_img[old_path] = image_load_only(old_path)
                self.data.append({'data': [preload_img[young_path], preload_img[old_path]]})
            
            self.augmentation = only_resize()
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        young_path, old_path = self.data[index]['data']
        young_img, old_img = image_agmentation_apply(young_path, old_path, self.augmentation)
        
        return young_img, old_img
    
    
# When there's not enough RAM
# class TestDataset(Dataset):
#     def __init__(self, img_path_for_glob, csv_path):
#         target_files = glob.glob(img_path_for_glob)
#         self.data = []
#         with open(csv_path, 'r', encoding='utf-8') as f:
#             csv_reader = csv.reader(f)
#             csv_reader = list(csv_reader)
#             for line in csv_reader[1:]:
#                 _, young, old = line
#                 for file_name in target_files:
#                     if young in file_name:
#                         young_path = file_name
#                     if old in file_name:
#                         old_path = file_name
#                 self.data.append({'data': [young_path, old_path]})
            
#             self.augmentation = only_resize()
            
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         young_path, old_path = self.data[index]['data']
#         young_img, old_img = image_load(young_path, old_path, self.augmentation)
        
#         return young_img, old_img

In [ ]:
BATCH_SIZE = 1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TEST_CSV_NAME = 'test_data.csv'
SAVE_FOLDER = 'save_model'
CSV_SAVE_FOLDER = 'predicted_csv'
os.makedirs(CSV_SAVE_FOLDER, exist_ok=True)

test_dataset = TestDataset(img_path_for_glob=os.path.join(BASE_DATA_ROOT, TEST_DATA_ROOT, '*', '*', '*'), csv_path=os.path.join(BASE_DATA_ROOT, TEST_CSV_NAME))
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, prefetch_factor=BATCH_SIZE*2, pin_memory=True)

model = PlantAgePrediction().to(DEVICE)
for model_num in range(1, 1000): # the range function can be changed with tuple or list with numbers. These numbers are saved model weights file names.
    MODEL_NUMBER = str(model_num)
    checkpoint = torch.load(SAVE_FOLDER + '/' + MODEL_NUMBER + '.pt', map_location=DEVICE)

    for key in list(checkpoint.keys()):
        if 'module.' in key:
            checkpoint[key.replace('module.', '')] = checkpoint[key]
            del checkpoint[key]
    model.load_state_dict(checkpoint)
    del checkpoint
    step = 0
    model.eval()
    total_loss = 0
    with torch.no_grad():
        
        with open(CSV_SAVE_FOLDER + '/' + MODEL_NUMBER + '.csv', 'w', encoding='utf-8') as f: # predicted results are saved CSV_SAVE_FOLDER.
            csv_writer = csv.writer(f)
            temp = ['idx','time_delta']
            csv_writer.writerow(temp)
            for data in tqdm.tqdm(test_dataloader):
                young, old = data
                young = young.to(DEVICE)
                old = old.to(DEVICE)
                logits = model(young, old)
                temp = [step, max(1, denormalize(float(logits.detach().cpu().numpy()), MINMAX_MIN, MINMAX_MAX))]
                csv_writer.writerow(temp)
                step += 1
            

이것으로 종료되었습니다.

GPU의 한계와 개인적인 일들이 있어 시간을 많이 투자 할 수 없어서 아쉬움은 있습니다.

가장 어려웠던 점은 validation 결과와 실제로 리더보드의 결과가 매우 불일치하는 경우들이 있었던 것입니다. 제가 제출한 결과 중 제일 성능이 좋았던 것 또한 해당 모델 훈련 과정 중 validation 결과 상에서 최고의 결과가 아니라 상위 4번째의 것이었습니다. validation set이 적기 때문에 벌어진 일이었을 수는 있으나 상당히 감을 잡기 난해하였습니다.

공부가 제일 큰 목적이었기에, 모델 아키텍쳐 구성, 파라미터 튜닝, 데이터셋 처리 등에 초점을 두다보니 앙상블은 시도하지 못한게 아쉽네요. k-fold 형식을 도입했었어도 다소 향상이 있을 수 있지 않았을까 합니다.

GPU 문제를 제외하고, 제가 생각했지만 시간 상 시도하지 못했던 것이 있습니다.

현재 모델은 투입되는 이미지 크기의 20%(최소 식물 하나는 보일 수 있는 크기)부터 100%(전체 이미지)까지 다양한 크기로 crop하여 448*448 크기로 리사이즈하여 학습하게 되었습니다.

다만,  원래 제가 세운 가설로는 하나의 식물이 지니는 나이를 유추할 수 있는 feature를 모델이 학습하고 이를 통해서 결과값을 산출 할 수 있다는 것이었고, 특히 하나의 이미지에 항상 식물이 5개가 있었으므로, 다양한 크기로 crop하여 학습하더라도 5개의 식물로부터 평균내어 정확한 결과값을 산출할 수 있을거라고 기대하였습니다.

그런데 대회 하루 앞두고 들었던 생각으로, 근본적인 문제가 전체 이미지가 resize되면서 세부적인 이미지 정보가 사라질 수 있다는 점입니다. 그러면 제가 생각했던 가설이 근본부터 틀리게 되는 결과를 낳지 않았을까 합니다

따라서 추후 제안해보고 싶은 것은, 이미지를 20%~40% 크기로 크롭하여 식물의 세밀한 feature를 모델이 충분히 학습할 수 있도록한 뒤, TEST 시점에서는 주어진 이미지를 4분할, 혹은 식물이 있는 곳을 중심으로 일부 중첩되더라도 5분할로 하여 이에 대한 결과값들을 평균내는 식으로 하면 정확도가 더 오를 수 있지 않았을까 합니다.

아쉽지만 여기서 마무리 하겠습니다. 모두 수고하셨습니다.